# Multivariate Poisson using hybrid MM-initial + Newton's for joint estimation

Now, Instead of doing block update iterating between Newton and MM-Algorithm, we will try initializing variance components using the MM-algorithm conditional on the independent mean effects and then run Newton's method jointly.
    
   ### Simulation Parameters (Truth):
   $$\mu = 5.000, vc_1 = 0.100, vc_2 = 0.100$$
   
   ### Using initial MM + Newton + mehrotra: (21 iterations, 22.5 seconds)
   $$\mu = 5.004, vc_1 = 0.110, vc_2 = 0.115$$
    

In [1]:
using GLMCopula, Random, Statistics, Test, LinearAlgebra, StatsFuns, GLM, Revise

In [2]:
Random.seed!(1234)
n = 50
variance_component_1 = 0.1
variance_component_2 = 0.1
Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)

mean = 5
dist = Poisson
vecd = [dist(mean) for i in 1:n]
    
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

NonMixedMultivariateDistribution{Discrete,Poisson{Float64},Array{Poisson{Float64},1}}(Poisson{Float64}[Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0)  …  Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0), Poisson{Float64}(λ=5.0)], [0.2 0.1 … 0.1 0.1; 0.1 0.2 … 0.1 0.1; … ; 0.1 0.1 … 0.2 0.1; 0.1 0.1 … 0.1 0.2], 9.999999999999996, Union{ContinuousUnivariateCopula, DiscreteUnivariateCopula}[#undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef  …  #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef])

In [3]:
nsample = 10_000
Random.seed!(1234)
@time Y_nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, nsample)

  4.231337 seconds (10.14 M allocations: 7.823 GiB, 18.13% gc time)


10000-element Array{Array{Float64,1},1}:
 [7.0, 8.0, 7.0, 4.0, 8.0, 2.0, 6.0, 6.0, 6.0, 7.0  …  5.0, 3.0, 4.0, 6.0, 4.0, 5.0, 5.0, 6.0, 4.0, 7.0]
 [3.0, 7.0, 6.0, 3.0, 6.0, 6.0, 3.0, 3.0, 8.0, 5.0  …  3.0, 4.0, 5.0, 8.0, 2.0, 5.0, 4.0, 4.0, 8.0, 6.0]
 [6.0, 3.0, 3.0, 7.0, 4.0, 3.0, 6.0, 2.0, 1.0, 8.0  …  2.0, 4.0, 3.0, 4.0, 6.0, 3.0, 4.0, 4.0, 3.0, 5.0]
 [9.0, 4.0, 5.0, 3.0, 5.0, 5.0, 4.0, 12.0, 4.0, 5.0  …  4.0, 8.0, 7.0, 9.0, 5.0, 3.0, 8.0, 5.0, 4.0, 4.0]
 [5.0, 7.0, 4.0, 7.0, 4.0, 1.0, 7.0, 3.0, 5.0, 4.0  …  9.0, 7.0, 3.0, 1.0, 3.0, 5.0, 3.0, 6.0, 5.0, 5.0]
 [6.0, 6.0, 10.0, 2.0, 4.0, 2.0, 6.0, 7.0, 1.0, 5.0  …  2.0, 6.0, 3.0, 6.0, 3.0, 6.0, 4.0, 4.0, 4.0, 5.0]
 [5.0, 4.0, 6.0, 3.0, 4.0, 6.0, 0.0, 3.0, 3.0, 3.0  …  4.0, 6.0, 3.0, 3.0, 5.0, 1.0, 4.0, 1.0, 5.0, 10.0]
 [6.0, 4.0, 5.0, 7.0, 5.0, 8.0, 5.0, 3.0, 10.0, 1.0  …  5.0, 4.0, 5.0, 3.0, 4.0, 6.0, 5.0, 4.0, 7.0, 2.0]
 [4.0, 5.0, 8.0, 5.0, 5.0, 4.0, 7.0, 3.0, 3.0, 6.0  …  5.0, 2.0, 4.0, 7.0, 6.0, 5.0, 6.0, 5.0, 7.0, 4.0]
 [4.0, 6.0

In [4]:
d = Poisson()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, nsample)
for i in 1:nsample
    y = Float64.(Y_nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = GLMCopulaVCObs(y, X, V, d, link)
end
gcm = GLMCopulaVCModel(gcs);

initialize_model!(gcm)
@show gcm.β
@show gcm.Σ

initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -1.1106679665667238e6 39999
2 -1.1106679665667238e6 -1.1106679665667238e6 39999
initializing variance components using MM-Algorithm
gcm.β = [1.6140126324276531]
gcm.Σ = [0.09215963543599975, 0.09063686128269058]


2-element Array{Float64,1}:
 0.09215963543599975
 0.09063686128269058

In [5]:
@test loglikelihood!(gcm, true, true) ≈ -1.1093040785410171e6

Test Passed

## Newton using Hessian + merotra

In [6]:
@time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100, mehrotra_algorithm = "yes", warm_start_init_point = "yes", hessian_approximation = "exact"))


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:        3
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equal

-1.1092926528694876e6

In [7]:
@show GLMCopula.loglikelihood!(gcm, true, true);

GLMCopula.loglikelihood!(gcm, true, true) = -1.1092926528694876e6


In [8]:
@show gcm.θ;

gcm.θ = [1.6102599493811351, 0.10979563246169695, 0.11466180111296703]


In [9]:
@show gcm.∇θ;

gcm.∇θ = [-1.987323727803414e-7, 8.147253538481891e-9, -1.0995790944434702e-8]


In [10]:
println("estimated mean = $(exp(gcm.β[1])); true mean value= $mean")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2");

estimated mean = 5.004111874560031; true mean value= 5
estimated variance component 1 = 0.10979563246169695; true variance component 1 = 0.1
estimated variance component 2 = 0.11466180111296703; true variance component 2 = 0.1


In [11]:
using BenchmarkTools

@benchmark loglikelihood!($gcm, true, true)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     722.928 ms (0.00% GC)
  median time:      724.976 ms (0.00% GC)
  mean time:        726.487 ms (0.00% GC)
  maximum time:     733.462 ms (0.00% GC)
  --------------
  samples:          7
  evals/sample:     1